# CNMF-E demo pipeline
##### version 2025, modified by Eunju Sung

This notebook demonstrates how to use Caiman for processing 1p microendoscopic data. It shows how to use Caiman for the following steps:

1. Apply the nonrigid motion correction (NoRMCorre) algorithm for motion correction to the original movie.
2. Apply the constrained nonnegative matrix factorization endoscopic (CNMF-E) source separation algorithm to extract an initial estimate of neuronal spatial footprint and calcium traces.
3. Apply quality control metrics to evaluate the initial estimates to narrow them down to a final set of estimates.

\+ plot and save extracted dF/F trace and clustered heatmap

reference: https://github.com/flatironinstitute/CaImAn/blob/main/demos/notebooks/demo_pipeline_cnmfE.ipynb

or see CaImAn docs for more information: https://caiman.readthedocs.io/en/latest/index.html

# Imports and general setup

In [ ]:
import bokeh.plotting as bpl
import cv2
import glob
import holoviews as hv
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.utils.visualization import view_quilt

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
        # get_ipython().run_line_magic('matplotlib', 'qt')  #uncomment to run in qt mode
except NameError:
    pass

try:
    cv2.setNumThreads(0)
except:
    pass

bpl.output_notebook()
hv.notebook_extension('bokeh')

Set up logger and environment variables

In [ ]:
# set up logging
logfile = None # Replace with a path if you want to log to a file
logger = logging.getLogger('caiman')
# Set to logging.INFO if you want much output, potentially much more output
logger.setLevel(logging.WARNING)
logfmt = logging.Formatter('%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s')
if logfile is not None:
    handler = logging.FileHandler(logfile)
else:
    handler = logging.StreamHandler()
handler.setFormatter(logfmt)
logger.addHandler(handler)

# set env variables in case they weren't already set
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"

# Set file path and shading function

In [ ]:
# Set desired filename
movie_filename = "movement_correction"

movie_path = [f'{movie_filename}.tif']
output_path = 'result'
os.makedirs(output_path, exist_ok=True)

In [ ]:
print(f"You have {psutil.cpu_count()} CPUs available in your current environment")
num_processors_to_use = None

If you have already set up multiprocessing (the `cluster` variable is already in your namespace), then that cluster will be closed and a new one created.

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'cluster' in locals():  # 'locals' contains list of current local variables
    print('Closing previous cluster')
    cm.stop_server(dview=cluster)
print("Setting up new cluster")
_, cluster, n_processes = cm.cluster.setup_cluster(backend='multiprocessing', 
                                                 n_processes=num_processors_to_use, 
                                                 ignore_preexisting=False)
print(f"Successfully set up cluster with {n_processes} processes")

# Define parameters
We first set some parameters related to the data and motion correction and create a `params` object. We'll modify this parameter object later on with settings for source extraction. You can also set all the parameters at once as demonstrated in the `demo_pipeline.ipynb` notebook.

Note here we are setting `pw_rigid` to `False`, as our data seems to mainly contain large-scale translational motion. We can always redo this later if it turns out to be a mistake.

# Key parameters for CNMF...
#### 1. rf (int): patch half-width

rf ('receptive field') is the half width of patches that in pixels (the actual patch width is 2*rf + 1). See previous image for a representation of how the field of view is split up into patches for parallel procesing. rf should be at least 3-4 times larger than the observed neuron diameter. The larger the patch size, the less parallelization will be used by Caiman. If rf is set to None, then CNMF will be run on the entire field of view.

#### 2. stride (int): patch overlap

stride is the overlap between patches in pixels (the actual overlap is stride + 1). This should be at least the diameter of a neuron. The larger the overlap, the greater the computational load, but the results will be more accurate when stitching together results from different patches.

#### 3. K (int): components per patch

K is the expected number of components per patch. You should adapt this to the density of components in your data, and the current rf parameter. We suggest you pick K based on the more dense patches in your movie so you don't miss neurons (we want to avoid false negatives).

#### 4. gSig (int, int): half-width of neurons

gSig is roughly the half-width of neurons in your movie in pixels (height, width): it is the sigma parameter of a Gaussian filter run on all the images during initialization. If the filter matches the neurons, you will get a much better estimate. gSig goes with gSiz, which is the kernel size (height and width in pixels) used for the filter. See the GaussianBlur() OpenCV function for more details on the sigma and size parameters.

merge_thr (float): merge threshold

If two spatially overlapping components are correlated above merge_thr, they will be merged into one component. The correlation coefficient is calculated using their calcium traces. If Caiman identifies a "component" that clearly contains two overlapping components, then increase merge_thr.

You typically will set rf and stride infrequently, so K, gSig, and merge_thr are the main parameters you will tweak when analyzing a given session. Note these are not the only important parameters. They just tend to be the most important: the others tend to depend on your calcium indicator or other factors that don't vary within an experimental session.


### all parameters
- `fnames`: List of paths to the file(s) to be analysed. Memmap and hdf5 result files will be saved in the same directory.
- `fr`: Imaging frame rate in frames per second.
- `decay_time`: Length of a typical transient in seconds. decay_time is an approximation of the time scale over which to expect a significant shift in the calcium signal during a transient. It defaults to 0.4, which is appropriate for fast indicators (GCaMP6f), slow indicators might use 1 or even more. However, decay_time does not have to precisely fit the data, approximations are enough.
- `p`: Order of the autoregressive model. p = 0 turns deconvolution off. If transients in your data rise instantaneously, set p = 1 (occurs at low sample rate or slow indicator). If transients have visible rise time, set p = 2. If the wrong order is chosen, spikes are extracted unreliably.
- `nb`: Number of global background components. This is a measure of the complexity of your background noise. Defaults to nb = 2, assuming a relatively homogeneous background. nb = 3 might fit for more complex noise, nb = 1 is usually too low. If nb is set too low, extracted traces appear too noisy, if nb is set too high, neuronal signal starts getting absorbed into the background reduction, resulting in reduced transients.
- `merge_thr`: Merging threshold of components after initialization. If two components are correlated more than this value (e.g. when during initialization a neuron was split in two components), they are merged and treated as one.
- `rf`: Half-size of the patches in pixels. Should be at least 3 to 4 times larger than the expected neuron size to capture the complete neuron and its local background. Larger patches lead to less parallelization.
- `stride`: Overlap between patches in pixels. This should be roughly the neuron diameter. Larger overlap increases computational load, but yields better results during reconstruction/denoising of the data.
- `K`: Number of (expected) components per patch. Adapt to rf and estimated component density.
- `gSig`: Expected half-size of neurons in pixels [rows X columns]. CRUCIAL parameter for proper component detection.
- `gSig_filt`: The size of the kernel is given from the parameter `gSig_filt`. Motion correction performs the best when `gSig_filt` is same with `gSig`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
- `method_init`: Initialization method, depends mainly on the recording method. Use greedy_roi for 2p data, corr_pnr for 1p data, and sparse_nmf for dendritic/axonal data.
- `ssub/tsub`: Spatial and temporal subsampling during initialization. Defaults to 1 (no compression). Can be set to 2 or even higher to save resources, but might impair detection/extraction quality.


In [ ]:
# dataset dependent parameters
frate = 10                    # movie frame rate
decay_time =                  # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = True          # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (,)          # sigma for high pass spatial filter applied before motion correction, used in 1p data (half-size of neurons)
max_shifts = (,)         # maximum allowed rigid shift
strides = (40, 40)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (20, 20)      # overlap between patches (size of patch = strides + overlaps)
max_deviation_rigid = 5  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries

mc_dict = {
    'fnames': movie_path,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan
}

parameters = params.CNMFParams(params_dict=mc_dict)

# Motion Correction
The background signal in micro-endoscopic data is very strong and makes motion correction challenging. As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the lower-frequency background activity and enhance spatial landmarks. The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no preprocessing is performed (default option, used in 2p data for CNMF). 

After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data, not the preprocessed data, so no information is lost before source separation. The motion corrected files are saved in memory mapped format. If no motion correction is performed (i.e., `motion_correct` was set to `False`), then the file gets directly memory mapped.

> For a more detailed exploration of Caiman's motion correction pipeline, see `demo_motion_correction.ipynb`. 

The following also plots the discovered displacements in x- and y- directions.

In [ ]:
%%time

%matplotlib inline
if motion_correct:
    # do motion correction rigid
    mot_correct = MotionCorrect(movie_path, dview=None, **parameters.get_group('motion'))
    mot_correct.motion_correct(save_movie=True)
    fname_mc = mot_correct.fname_tot_els if pw_rigid else mot_correct.fname_tot_rig
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mot_correct.x_shifts_els)),
                                     np.max(np.abs(mot_correct.y_shifts_els)))).astype(int)
    else:
        bord_px = np.ceil(np.max(np.abs(mot_correct.shifts_rig))).astype(int)

    bord_px = 0 if border_nan == 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(movie_path, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)

In [ ]:
%matplotlib inline

# plot shifts
plt.plot(mot_correct.shifts_rig)
plt.legend(['x shifts', 'y shifts'])
plt.xlabel('frames')
plt.ylabel('pixels')
plt.gcf().set_size_inches(6,3)

plt.savefig(f"{output_path}/motion_shifts.png")
plt.show()
shift_distance = []
for rig in mot_correct.shifts_rig:
    shift_distance.append(np.sqrt(np.power(rig[0],2)+np.power(rig[1],2)))
plt.clf()

# plot distance of shifts
plt.plot(shift_distance, color = 'black', linewidth = 0.5) 
plt.legend(['shift_distance'])
plt.xlabel('frames')
plt.ylabel('pixels')
plt.gcf().set_size_inches(6,3)
plt.savefig(f"{output_path}/motion_distance.png")
plt.show()


## Load memory mapped file
Memory mapping is discussed in more detail in `demo_pipeline.ipynb`.

In [ ]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [ ]:
from tifffile import imsave
from skimage import io

mot_corrected = np.array(images)
io.imsave(os.path.join(output_path, "motion_corrected_movie.tif"), mot_corrected)


## Inspect summary images and set parameters
### Correlation-pnr plot
For CNMFE, Caiman uses the correlation and peak-to-noise ratio (PNR) for initialization, which will both tend to be high in regions that contain neurons. Hence, we set a threshold for both quantitites to remove the low correlation/low pnr regions and seed initialization with regions most likely to contain neuronal activity. 

Using `nb_inspect_correlation_pnr()`, you can inspect the correlation and PNR images to find threshold values for these quantities: `min_corr` and `min_pnr`. You can adjust the range of values displayed in the two subplots below by choosing the Y-box select tool (third button from the left) and selecting the desired region in the histograms on the right hand side of each image. You can also use the pan button (first button on the left) to zoom/adjust the axis limits in the histogram to make it easier to see the limits.

In [ ]:
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline

# Subsample and inspect correlation/PNR
correlation_image, peak_to_noise_ratio = cm.summary_images.correlation_pnr(
    images[::max(T // 1000, 1)],  # subsample if T is large
    gSig=5,                       # filter size
    swap_dim=False                # fix orientation issue if needed
)

# Bokeh-based interactive viewer
nb_inspect_correlation_pnr(
    correlation_image,
    peak_to_noise_ratio,
    cmap='inferno'  # jet, fire, viridis are also good
)


# Parameter setting for CNMF-E
Everything is now set up to run source extraction with CNMFE. We will construct a new parameter dictionary and use this to modify the *existing* `parameters` object, using the `change_params()` method.

There are *two* main differences between the CNMF and CNMFE source separation algorithms. The first is the background model (this is discussed in the sidebar below on the Ring Model). The second difference is in how the models are initialized. This is addressed below when we go over setting corr/pnr thresholds for initialization, which we did not have to do for our 2p data.

For now, note that we have set `gnb` to `0`: this is effectively the flag telling Caiman to use CNMFE instead of CNMF. 

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None for CNMFE
gSig = np.array([,])  # expected half-width of neurons in pixels 
gSiz = 2*gSig + 1     # half-width of bounding box created around neurons during initialization
merge_thr = .6      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 10    # amount of overlap between the patches in pixels 
tsub = 1            # downsampling factor in time for initialization, increase if you have memory problems1
ssub = 1            # downsampling factor in space for initialization, increase if you have memory problems
gnb = 0             # number of background components (rank) if positive, set to 0 for CNMFE
low_rank_background = None  # None leaves background of each patch intact (use True if gnb>0)
nb_patch = 0        # number of background components (rank) per patch (0 for CNMFE)
min_corr =       # min peak value from correlation image
min_pnr =        # min peak to noise ration from PNR image
ssub_B = 1          # additional downsampling factor in space for background (increase to 2 if slow)
ring_size_factor = 1.5  # radius of ring is gSiz*ring_size_factor

parameters.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # True for 1p
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px});                # number of pixels to not consider in the borders)

Initialize the model using the parameters.

In [ ]:
cnmfe_model = cnmf.CNMF(n_processes=n_processes, 
                        dview=None, 
                        params=parameters)

<div class="alert alert-info">
    <h2 >CNMF-E: The Ring Model</h2>
   Background activity is very ill-behaved with 1p recordings: it often fluctuates locally and is much larger in magnitude than the neural signals we want to extract. In other words, the large-scale background model used for CNMF is not sufficient for most 1p data. Hence, Pengcheng Zhou and others came up with a localized model of background activity for CNMFE: the background at each pixel is represented as the weighted sum of activity from a circle (or ring) of pixels a certain distance from that pixel. The distance of this ring from the reference pixel is set by the <em>ring_size_factor</em> parameter. This more complex pixel-by-pixel background model explains why CNMFE is computationally more expensive than CNMF, and also why it works better to mop up large-scale localized background noise in  1p data. 
    
<p>When you set <em>gnb</em> in the CNMF model (usually to 1 or 2), you are setting the number of global background components. The fact that you can get away with so few is testament to how well-behaved the background activity is in 2p recordings. When we set <em>gnb</em> to 0 in Caiman, this is a flag telling Caiman's back end to switch to the more complicated ring model of the background activity.</p>

For more details on CNMFE you can see the <a href="https://elifesciences.org/articles/28728">original paper</a> and the <a href="https://elifesciences.org/articles/38173">Caiman paper</a>. 
</div>

## Key parameters for CNMFE
The key parameters for CNMFE are slightly different than for CNMF, but with some overlap. As we'll see, because of the high levels of background activity, we can't initialize the same way as with CNMF. We have two new important parameters directly related to initialization that come into play: `min_corr` and `min_pnr`. 

`rf` (int): *patch half-width*
> `rf`, which stands for 'receptive field', is the half width of patches in pixels. The patch width is `2*rf + 1`. `rf` should be *at least* 3-4 times larger than the observed neuron diameter. The larger the patch size, the less parallelization will be used by Caiman. If `rf` is set to `None`, then CNMFE will be run on the entire field of view.

`stride` (int): *patch overlap*
> `stride` is the overlap between patches in pixels (the actual overlap is `stride_cnmf + 1`). This should be at least the diameter of a neuron. The larger the overlap, the greater the computational load, but the results will be more accurate when stitching together results from different patches. This param should probably have been called 'overlap' instead of 'stride'.

`gSig (int, int)`: *half-width of neurons*
> `gSig` is roughly the half-width of neurons in your movie in pixels (height, width). It is the standard deviation of the mean-centered Gaussian used to filter the movie before initialization for CNMFE. It is related to the `gSiz` parameter, which is the width of the entire kernel filter.

`merge_thr (float)`: *merge threshold* 
> If the correlation between two spatially overlapping components is above `merge_thr`, they will be merged into one component. 

`min_corr` (float): *minimum correlation*
> Pixels from neurons tend to be correlated with their neighbors. During initialization, Caiman filters out those pixels below `min_corr` to help select seed pixels. We discuss this more below.

`min_pnr` (float): *minimum peak to noise ratio*
> Pixels from neurons tend to have a high signal-to-noise ratio. During initialization, Caiman filters out those pixels below `min_pnr` to help select seed pixels. We discuss this more below.

<div class="alert alert-info">
    <h2>CNMFE initialization: More on correlation and peak-to-noise-ratio</h2>
     <img src="images/mn_centered_gaussian.jpg" align="right" width="200"></img>
How are correlation and peak-to-noise ratio actually calculated? First Caiman convolves the motion corrected movie with a <i>mean-centered Gaussian</i> (example to the right). The sigma of the Gaussian is <em>gSig</em>, and mean centering is turned on by setting <em>center_psf</em> to <em>True</em>. This mean centering creates a Gaussian with a positive peak in the middle of width <i>approximately</i> <em>gSig/2</em>, surrounded by a negative trench, and a ledge of zeros around the outer edges. This crucial preprocessing filter serves to highlight neuronal peaks and smooth away low-frequency background activity.

<p>The function <em>correlation_pnr()</em> applies this mean-centered Gaussian to each frame of the motion corrected movie and returns the correlation image of that movie, as well as the peak-to-noise-ratio (PNR). The correlation image is the correlation of each pixel with its neighbors. The PNR is the ratio of the maximum magnitude at a pixel to the noise value at that pixel (it is a fast and rough measure of signal-to-noise). As mentioned above, both of these values tend to be higher in pixels that contain neurons. The CNMFE initialization procedure is to set a threshold for both quantities, take their <i>product</i>, and use the peaks in this product map to find <b>seed pixels</b> for initialization of the CNMFE source separation algorithm.</p>

More details on the initialization procedure used here can be found in the <a href="https://elifesciences.org/articles/28728">CNMFE paper</a>, or by exploring the code.         
</div>

## Run the CNMF-E algorithm

In [ ]:
cnmfe_model.fit(images);

Running the algorithm creates an `estimates` class, which we discuss in detail in `demo_pipeline.ipynb`. The CNMFE `estimates` class includes almost all the same attributes as with CNMF, such as the neural spatial and temporal components `A` and `C`. 

It also includes the discovered model of background activity, which is different from the CNMF model. For *CNMF* the background model is returned as low-rank matrices `b` and `f`. For CNMFE, the background model parameters are represented in the matrix `W` (the weights of the *ring model* -- discussed above -- for each pixel) as well as `b0` (a constant offset for each pixel). We will show how to reconstruct the background activity below. 

# Component Evaluation
Source extraction typically produces many false positives. Our next step is quality control: separating the results into "good" and "bad" neurons using two different metrics (discussed in detail in `demo_notebook.ipynb`):

- **Signal-to-noise ratio (SNR)**: a minimum SNR is set for the calcium transients (`min_SNR`).
- **Spatial correlation**:  a minimum correlation is set between the shape of each component and the frames in the movie when that component is active (`rval_thr`). 

> Caiman does *not* use the CNN classifier to sort neurons based on shape for 1p data: the network was trained on 2p data. Hence, we set the `use_cnn` param to `False`. 

Here we set the two parameters and run `evaluate_components()` to see which pass muster:

In [ ]:
min_SNR = 10            # SNR threshold
rval_thr = 0.7 # spatial correlation threshold

quality_params = {'min_SNR': min_SNR,
                  'rval_thr': rval_thr,
                  'use_cnn': False}
cnmfe_model.params.change_params(params_dict=quality_params)

cnmfe_model.estimates.evaluate_components(images, cnmfe_model.params, dview=cluster)

print('*****')
print(f"Total number of components: {len(cnmfe_model.estimates.C)}")
print(f"Number accepted: {len(cnmfe_model.estimates.idx_components)}")
print(f"Number rejected: {len(cnmfe_model.estimates.idx_components_bad)}")

# Visualize results

In [ ]:
#%% plot contour plots of accepted and rejected components
cnmfe_model.estimates.plot_contours_nb(img=correlation_image, 
                                       idx=cnmfe_model.estimates.idx_components);

View traces of accepted and rejected components:

In [ ]:
cnmfe_model.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmfe_model.estimates.idx_components,
                                        cmap='viridis', #gray
                                        thr=.9); #increase to see full footprint

# Save and load results

#### The results of Caiman are saved in an estimates object. This is stored inside the cnmf object, i.e. it can be accessed using cnmf.estimates. The variables of interest are:

`estimates.A`: Set of spatial components. Saved as a sparse column format matrix with dimensions (# of pixels X # of components). Each column corresponds to a spatial component.

`estimates.C`: Set of temporal components. Saved as a numpy array with dimensions (# of components X # of timesteps). Each row corresponds to a temporal component denoised and deconvolved.

`estimates.b`: Set of background spatial components (for 2p analysis): Saved as a numpy array with dimensions (# of pixels X # of components). Each column corresponds to a spatial background component.

`estimates.f`: Set of temporal background components (for 2p analysis). Saved as a numpy array with dimensions (# of background components X # of timesteps). Each row corresponds to a temporal background component.

`estimates.S`: Deconvolved neural activity (spikes) for each component. Saved as a numpy array with dimensions (# of background components X # of timesteps). Each row corresponds to the deconvolved neural activity for the corresponding component.

`estimates.YrA`: Set of residual components. Saved as a numpy array with dimensions (# of components X # of timesteps). Each row corresponds to the residual signal after denoising the corresponding component in estimates.C.

`estimates.F_dff`: Set of DF/F normalized temporal components. Saved as a numpy array with dimensions (# of components X # of timesteps). Each row corresponds to the DF/F fluorescence for the corresponding component.

`estimates.sn`: Standard deviation of the noise distribution. If no value is given, then sn is estimated from the data.

`estimates.bl`: Fluorescence baseline value. If no value is given, then bl is estimated from the data.

`estimates.b0`: The variables for one photon processing, for the baseline value for each pixel.

`dF_F`: calculated delta F over F. (caution: original CaIMaN strongly recommend NOT TO use dF/F for 1p image, because the background signal is ill-behaved.

In fact, `dF_F`, `estimates.C`, and `detrended calcium trace` all show the same graph. Only difference is *unit* of the y axis.

See below:

### Extract $\Delta F/F$ values
Currently in Caiman, we don't return a true dff value for 1p data. This is because, as mentioned in `demo_pipeline.ipynb`,  Caiman normalizes to both the baseline fluorescence and background activity. The background activity in 1p is so ill-behaved (as discussed above in the sidebar on the ring model) that Caiman currently only *detrends* the data by subtracting away the baseline but not normalizing the data. This explains the warning you will see when you run the following:


    cnmfe_model.estimates.detrend_df_f(quantileMin=8, 
                                      frames_window=250,
                                      flag_auto=False,
                                      use_residuals=False);  # use denoised data

In [ ]:
from caiman.source_extraction.cnmf.utilities import extract_DF_F, detrend_df_f
import numpy as np
from scipy.interpolate import interp1d
from skimage.draw import polygon

from caiman.source_extraction.cnmf.utilities import extract_DF_F

index_list = cnmfe_model.estimates.idx_components
index_list_bad = cnmfe_model.estimates.idx_components_bad

C = cnmfe_model.estimates.C
YrA = cnmfe_model.estimates.YrA
f = cnmfe_model.estimates.f
A = cnmfe_model.estimates.A
b = cnmfe_model.estimates.b
bl = cnmfe_model.estimates.bl
b0 = cnmfe_model.estimates.b0
sn = cnmfe_model.estimates.sn
dims = cnmfe_model.estimates.dims

## extract df/f by robust calculation
epsilon = 1e-6
img_array = np.array(images)
t,y,x = img_array.shape
background_tot = cnmfe_model.estimates.compute_background(Yr)
background_tot = background_tot.reshape(img_array.shape[2],img_array.shape[1],background_tot.shape[1])
signal=np.zeros((len(C),t))
background=np.zeros((len(C),t))

for index in index_list:
    coors=cnmfe_model.estimates.coordinates[index]
    x1,y1=zip(*coors['coordinates'])
    x1 = np.array(x1[1:-1])
    y1 = np.array(y1[1:-1])

    valid_mask = ~np.isnan(x1) & ~np.isnan(y1)
    x1 = x1[valid_mask]
    y1 = y1[valid_mask]

    xx1, yy1 = polygon(x1, y1)

    xx1[xx1 >= x] = x - 1
    yy1[yy1 >= y] = y - 1

    unique_coords = np.unique(np.column_stack((xx1, yy1)), axis=0)
    xx1 = unique_coords[:, 0]
    yy1 = unique_coords[:, 1]

    for t in range(img_array.shape[0]):
        signal[index][t] = np.mean(img_array[t,yy1, xx1])
        background[index][t]=np.mean(background_tot[xx1, yy1, t])

dF_F = (signal - background) / background

# Visualize accepted ROI coordinates 

In [ ]:
%matplotlib inline
import csv
import os

from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from skimage import io
n, y, x = images.shape

########## mapping of accepted neurons ############

fig, axes = plt.subplots(1,2, figsize = (32,13))
axes[0].imshow(correlation_image, cmap='gray')
size = 0
for i in index_list:
    rcolor = np.random.rand(3,)
    axes[0].plot(cnmfe_model.estimates.coordinates[i]['coordinates'][1:-1,0],cnmfe_model.estimates.coordinates[i]['coordinates'][1:-1,1],
                linewidth=3, color=rcolor)
    height = min(cnmfe_model.estimates.coordinates[i]['coordinates'][1:-1,1])
    axes[0].text(max(cnmfe_model.estimates.coordinates[i]['coordinates'][1:-1,0])-10, height+10, i, ha='center', va='bottom', 
                size=15,color=rcolor)
    axes[1].plot(dF_F[i], alpha=0.6, color=rcolor)
    size += 1
#axes[0].legend(index_list, fontsize=10, bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[0].set_xlim([0,x])
axes[0].set_ylim([y,0])
axes[0].set_title("MAP of accepted ROIs", size=18, fontweight='heavy')
# axes[0].get_legend().remove()
# axes[1].legend(index_list, fontsize=10, bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[1].set_title("Calcium traces of accepted ROIs", size=18, fontweight='heavy')
axes[1].set_xlabel("frames",size=15)
axes[1].set_ylabel("dF/F",size=15)

plt.savefig(os.path.join(output_path, "accepted ROIs and Traces.png"))

# Plot clustering heatmap of df_f values

In [ ]:
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

data_for_clustering = accepted_df_f_pd.drop(columns=["time(sec)"]).T

# font size rescaling
sns.set_context("notebook", font_scale=0.4)
sns.set(style="white")




cg2 = sns.clustermap(
    data_for_clustering,
    method="ward",        # hierarchical clustering method
    metric="euclidean",   # distance metric
    cmap="inferno",
    figsize=(12, 10),
    col_cluster=False,     # 시간 축은 clustering하지 않음,
    vmax = 0.05
)
cg2.ax_heatmap.set_yticklabels([])
cg2.ax_heatmap.set_ylabel("")
cg2.ax_heatmap.tick_params(left=False, bottom=False)
time_values = data_for_clustering.columns
tick_step = 500
tick_indices = [i for i, t in enumerate(time_values) if t % tick_step == 0]

cg2.ax_heatmap.set_xticks(tick_indices)
cg2.ax_heatmap.set_xticklabels([f"{int(time_values[i]/frate)}" for i in tick_indices], rotation=90)
cg2.ax_heatmap.set_xlabel("Time (s)")

plt.title("Clustered Neural Activity (dF/F)")
plt.tight_layout()
plt.savefig(os.path.join(output_path, "clustered_neural_activity.png"))
plt.show()
